In [6]:
import os
import requests
from urllib.parse import urlparse

def active_enumerator(domain):
    """
    Enumerates subdomains for a given domain, and checks which ones are active.

    Args:
        domain (str): The domain to enumerate subdomains for.

    Returns:
        set: A set of subdomains.
    """
    # If the input is just a domain name without the protocol prefix, add "http://" to the beginning
    if not domain.startswith("http"):
        domain = "http://" + domain

    # Parse the domain to get just the hostname
    hostname = urlparse(domain).hostname

    subdomains = set()

    with open("../lists/subdomains", "r") as subdomain_file:
        for line in subdomain_file:
            subdomain = line.strip()
            if subdomain.endswith(hostname):
                subdomains.add(subdomain)

    active_subdomains = set()

    for subdomain in subdomains:
        try:
            response = requests.get(f"http://{subdomain}")
            if response.status_code < 400:
                active_subdomains.add(subdomain)
        except:
            pass

    return subdomains


In [9]:
active_enumerator("github.com")

FileNotFoundError: [Errno 2] No such file or directory: '../lists/subdomains'

In [10]:
import os

file_path = "../lists/subdomains"
print(os.path.isfile(file_path))

False
